# Flask REST API for Product and Category Management

## Project Overview
This project is a **RESTful API** built with Flask for managing products and their associated categories. It includes:
- **CRUD operations** for Products and Categories.
- **Swagger documentation** using Flask-RESTx.
- **SQLAlchemy** for ORM and database management.
- **Flask-Migrate** for handling schema migrations.
- **Environment variable support** with `python-dotenv`.

### API Overview

#### API Endpoints

##### 1. Products
- **`GET /products/`**: List all products.
- **`POST /products/`**: Create a new product.
- **`GET /products/<id>`**: Retrieve a product by ID.
- **`DELETE /products/<id>`**: Delete a product by ID.

##### 2. Categories
- **`GET /categories/`**: List all categories.
- **`POST /categories/`**: Create a new category.
- **`GET /categories/<id>`**: Retrieve a category by ID.
- **`DELETE /categories/<id>`**: Delete a category by ID.

---

#### API Models

##### 1. Product Model
- **`id`** *(Integer)*: Unique identifier of the product (read-only).
- **`name`** *(String)*: Name of the product (required).
- **`price`** *(Float)*: Price of the product (required).
- **`category`** *(Nested Category)*: The category associated with the product.

##### 2. Category Model
- **`id`** *(Integer)*: Unique identifier of the category (read-only).
- **`name`** *(String)*: Name of the category (required).
- **`description`** *(String)*: A brief description of the category.
## Project Structure
```plaintext
pythonFlaskProductCategory/
├── app/                   # Main app directory
│   ├── __init__.py        # Main app file with configurations
│   ├── models/            # Directory for database models
│   │   ├── __init__.py    # SQLAlchemy setup and model imports
│   │   ├── product.py     # Database model for Product
│   │   └── category.py    # Database model for Category
│   ├── schemas/           # Directory for Flask-RESTx schemas
│   │   ├── __init__.py    # Initialize schemas
│   │   ├── product_schema.py  # Schema for Product
│   │   └── category_schema.py # Schema for Category
│   ├── routes/            # Directory for API routes
│   │   ├── __init__.py    # Register routes function
│   │   ├── products.py    # Routes for Product API
│   │   └── categories.py  # Routes for Category API
│   ├── templates/         # Web views for interacting with the API
│       ├── base.html
│       ├── products.html
│       └── categories.html
├── static/                # Static files (CSS, JS, images)
│   ├── style.css
│   ├── script.js
├── migrations/            # Database migrations (created by Flask-Migrate)
├── .env                   # Environment variables
├── .venv/                 # Virtual environment directory
├── run.py                 # Entry point for running the application
└── requirements.txt       # Python dependencies
```

---

In [1]:
# models/__init__.py
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [3]:
# models/products.py
from app.models import db 

class Product(db.Model):
    __tablename__ = "products"
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(255), nullable=False)
    price = db.Column(db.Float, nullable=False)
    category_id = db.Column(db.Integer, db.ForeignKey('categories.id'))
    category = db.relationship("Category", back_populates='products' )
    

In [4]:
# models/categories.py
from app.models import db 

class Category(db.Model):
    __tablename__ = "categories"
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(255), nullable=False)
    description = db.Column(db.String(500), nullable=False)
    products = db.relationship('Product', back_populates='products')

In [13]:
# routes/__init__py

def register_routes(api):
    from app.routes.products import product_ns
    from app.routes.categories import category_ns
    
    api.add_namespace(product_ns)
    api.add_namespace(category_ns)

In [ ]:
# routes/products.py
from flask_restx import Namespace, Resource
from app.models.products import Product 
from app.models import db
from app.schemas.product_schema import product_model
 
product_ns = Namespace('products', description="Product operations")

@product_ns.route('/')
class ProductList(Resource):
    
    @product_ns.marshal_list_with(product_model)
    def get(self):
        """List all products"""
        return Product.query.all() # returns list of products 
       
    
    @product_ns.expect(product_model)
    def post(self):
        """Create a new product"""
        data = product_ns.payload
        new_product = Product(
                name=data['name'],
                price=data['price'],
                category_id=data['category']
            )
        db.session.add(new_product)
        db.session.commit()
        
        return new_product, 201

@product_ns.route('/<int: id>')
class ProductDetails(Resource):
    
    @product_ns.marshal_with(product_model)
    def get(self, id: int):
        """Retrieve a product by ID"""
        return Product.query.get_or_404(id)
    
    def delete(self, id: int):
        """Delete a product by ID"""
        product = Product.query.get_or_404(id) 
        db.session.delete(product)
        db.session.commit()
        
        return f"successfully deleted product with id {id}", 204

In [ ]:
# routes/categories.py
from flask_restx import Namespace, Resource
from app.models.categories import Category
from app.models import db
from app.schemas.category_schema import category_model
 
category_ns = Namespace('Categories', description='Category operations')

@category_ns.route('/')
class CategoryList(Resource):
    
    @product_ns.marshal_list_with(category_model)
    def get(self):
        """List all categories"""
        return Category.query.all()
    
    @category_ns.expect(category_model)
    def post(self):
        """Create a new category"""
        data = category_ns.payload
        new_category = Category(
            name=data["name"],
            description=data["description"]
        )
        db.session.add(new_category)
        db.session.commit()
        
        return new_category, 201
    
@product_ns.route('/<int: id>')
class CategoryDetail(Resource):
    
    @product_ns.marshal_with(category_model)
    def get(self, id: int):
        """Retrieve a category by ID"""
        return Category.query.get_or_404(id)
    
    def delete(self, id: int):
        """Delete a category by ID"""
        product = Category.query.get_or_404(id) 
        db.session.delete(product)
        db.session.commit()
        
        return f"successfully deleted category with id {id}", 204

In [8]:
# schemas/__init__.py
import os
from flask_restx import Api
  
app_title = os.getenv("APP_TITLE", "product and Category Management API")
app_version= os.getenv("APP_VERSION","1.0" )
app_description = os.getenv("APP_DESCRIPTION","API for managing products and categories.")

api = Api(
    title=app_title,
    version=app_version,
    description=app_description
)

In [9]:
# schemas/product_schema.py
from flask_restx import fields 
from app.schemas import api
""" 
READONLY MEANS = VALID ONLY FOR GET, PUT, DELETE  BUT NOT FOR POST
"""

product_model = api.model('Product', {
    'id': fields.Integer(readOnly=True, description='Unique identifier of the product'),
    'name': fields.String(required=True, description='Name of the product'),
    'price': fields.Float(required=True, description='Price of the product'),
    'category': fields.String(attribute='category.name', description='Category of the product')
})

In [10]:
# schemas/category_schema.py
from flask_restx import fields
from app.schemas import api

""" 
READONLY MEANS = VALID ONLY FOR GET, PUT, DELETE  BUT NOT FOR POST
"""
category_model = api.model('Category', {
    'id': fields.Integer(readOnly=True, description='Unique identifier of the category'),
    'name': fields.String(required=True, description='Name of the category'),
    'description': fields.String(description='Description of the category')
})

In [16]:
# app/__init__.py
import os
from flask import Flask
from flask_restx import Api, Namespace, Resource
from dotenv import load_dotenv

# Load environmental variables
load_dotenv()

def create_app():
    app = Flask(__name__)
    
    # Configuration
    
    app.config["SQLALCHEMY_DATABASE_URI"]= os.getenv("DATABASE_URI")
    app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
    
    
    # Initialize extensions
    from app.models import db
    db.init_app(app)
    
    from app.schemas import api 
    api.init_app(app)
    
    from app.routes import register_routes 
    register_routes(api)
    
    return app

In [ ]:
# run.py

import nest_asyncio
from werkzeug.serving import run_simple
from app import create_app

# Allow Flask to run in Jupyter
nest_asyncio.apply()

app = create_app()

run_simple('localhost', 5000, app)

if __name__ == "__main__":
    app.run(debug=True)
